In [1]:
import numpy as np
import cv2

## The Summary of AlexNet 

In [2]:
def alexnet_summary(net, layer_type=False, layer_shape=False, 
                    get_flops=False, get_memory=False,
                    get_total_time=False, get_layer_time=False):
    # check if the model is empty
    if net.empty():
        print("Net is empty")
        return
        
    # get the types of layers used in the model
    if layer_type:
        print("\nNet contains:")
        for t in net.getLayerTypes():
            print("\t{} layers of type {}".format(net.getLayersCount(t), t))
        
    
    
    # get the tensor shapes for the loaded model and specified input shape
    if layer_shape:
        layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
        layers_names = net.getLayerNames()
        print("\nNet layers shapes: ")
        for l in range(len(layers_names)):
            in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
            print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                                 in_num, out_num))
            for i in range(in_num):
                print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
            for i in range(out_num):
                print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
            
    # compute the number of FLOPs
    if get_flops:
        print("\ngflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    if get_memory:
        w, b = net.getMemoryConsumption((1, 3, 224, 224))
        print("\nweights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    if get_total_time:
        total, timings = net.getPerfProfile()
        tick2ms = 1000 / cv2.getTickFrequency()
        print("\ninference (ms): {:2f}".format(total * tick2ms))
    
    # report the per layer inference time
    if get_layer_time:
        layer_names = net.getLayerNames()
        print("\n{: <30} {}".format("LAYER", "TIME (ms)"))
        for (i,t) in enumerate(timings):
            print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [3]:
# read the model
model_path = "./model/model.onnx"
net = cv2.dnn.readNetFromONNX(model_path)    

In [4]:
alexnet_summary(net, layer_type=True)


Net contains:
	5 layers of type Convolution
	2 layers of type Dropout
	3 layers of type InnerProduct
	2 layers of type LRN
	3 layers of type Pooling
	7 layers of type Relu
	1 layers of type Reshape
	1 layers of type Softmax
	1 layers of type __NetInputLayer__


In [5]:
alexnet_summary(net, layer_shape=True)


Net layers shapes: 
  Layer conv1_1 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer conv1_2 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [ 1 96 54 54]
  Layer norm1_1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 96 54 54]
	output #0 has shape [ 1 96 54 54]
  Layer pool1_1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 96 54 54]
	output #0 has shape [ 1 96 54 54]
  Layer conv2_1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 96 54 54]
	output #0 has shape [ 1 96 26 26]
  Layer conv2_2 has 1 inputs and 1 outputs
	input #0 has shape [ 1 96 26 26]
	output #0 has shape [  1 256  26  26]
  Layer norm2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  26  26]
	output #0 has shape [  1 256  26  26]
  Layer pool2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  26  26]
	output #0 has shape [  1 256  26  26]
  Layer conv3_1 has 1 inputs and 1 outputs
	input #

In [6]:
alexnet_summary(net, get_flops=True, get_memory=True)


gflops:  2.1065083120000003

weights (mb): 243.860896 , blobs (mb): 7.804735999999999


In [7]:
alexnet_summary(net, get_total_time=True, get_layer_time=True)


inference (ms): 243.760466

LAYER                          TIME (ms)
conv1_1                        36.72
conv1_2                        0.00
norm1_1                        11.96
pool1_1                        3.39
conv2_1                        64.44
conv2_2                        0.00
norm2_1                        6.26
pool2_1                        1.06
conv3_1                        30.24
conv3_2                        0.00
conv4_1                        28.17
conv4_2                        0.00
conv5_1                        12.90
conv5_2                        0.00
pool5_1                        0.80
OC2_DUMMY_0                    0.01
fc6_1                          30.81
fc6_2                          0.00
fc6_3                          0.01
fc7_1                          14.73
fc7_2                          0.00
fc7_3                          0.01
fc8_1                          2.18
prob_1                         0.07
